# Algorithm Implementation
- problem : memory running out


In [20]:
#Libraries 
from keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.keras
from PIL import Image, ImageOps
from keras.preprocessing import image
import warnings
import gc
warnings.filterwarnings("ignore")

In [21]:
def base_models():
    vgg19 = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\vgg19.h5')
    naive = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\naive.h5')
    return vgg19,naive

In [22]:
#Model Value Analyser
def analyse(img_path):
    vgg19,naive = base_models()
    
    img = image.load_img(img_path, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    vgg19_score = vgg19.predict(img)
    
    img = image.load_img(img_path, target_size=(64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    naive_score = naive.predict(img)
    gc.collect()
    return [vgg19_score,naive_score]

In [23]:
def class_calculator(score_list):
    if score_list[0][0][0] > score_list[0][0][1]:
        vgg19_op = 0
    else:
        vgg19_op = 1
        
    if score_list[1][0][0] > score_list[1][0][1]:
        naive_op = 0
    else:
        naive_op = 1
        
    if vgg19_op == naive_op:
        gc.collect()
        return [vgg19_op,score_list] #further calculation
    else:
        gc.collect()
        return [5,score_list]

In [24]:
def second_eval(pred_class,img_path,score_list):
    img = image.load_img(img_path, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    if pred_class == 0:
        vgg16 = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\vgg16.h5')
        vgg16_score = vgg16.predict(img)
        final_score = (score_list[0][0][0]+score_list[1][0][0]+vgg16_score[0][0])/3
        gc.collect()
        return final_score
    elif pred_class == 1 :
        resnet50 = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\res50.h5')
        resnet50_score = resnet50.predict(img)
        final_score = (score_list[0][0][1]+score_list[1][0][1]+resnet50_score[0][1])/3
        gc.collect()
        return final_score
    else :
        tm = tensorflow.keras.models.load_model(r'D:\working repos\AltRekruit-Models\Models(15jan)\Models\tm_custom_model(image-pairs).h5')
        np.set_printoptions(suppress=True)
        tm.compile(loss='binary_crossentropy',
                      optimizer='rmsprop',
                  metrics=['accuracy'])
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        img = Image.open(img_path)
        size = (224, 224)
        img = ImageOps.fit(img, size, Image.ANTIALIAS)
        image_array = np.asarray(img)
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
        data[0] = normalized_image_array
    
        tm_score = tm.predict(data)
        
        if tm_score[0][0] > tm_score[0][1]:
            final_score = (score_list[0][0][0]+score_list[1][0][0]+tm_score[0][0])/3
            gc.collect()
            return final_score
        else:
            final_score = (score_list[0][0][1]+score_list[1][0][1]+tm_score[0][1])/3
            gc.collect()
            return final_score

In [25]:
img_path = r"D:\working repos\AltRekruit\Testing\Final (15jan)\images\good_1.jpg"
score_list = analyse(img_path)

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,256,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [ ]:
pred_class , score_list = class_calculator(score_list)
gc.collect()

In [ ]:
final_score = second_eval(pred_class,img_path,score_list)
gc.collect()

In [ ]:
final_score

Model_Files = http://bit.ly/alt_rekruit_models